## ML Baseline (АВТ): урезанный фичсет без групповой CV

Цель: получить сравнимую базовую линию для установок с названием, содержащим «АВТ»,
используя минимально достаточный набор признаков:
- геометрия: `cross_sectional_area`;
- материал: `material_resistance_score`;
- операционные: `operating_temperature`, `operating_pressure`;
- химические (2–3 ключевых): `sulfur_content`, `total_sulfur_compounds`, `h2s_content` (если доступны).

Здесь НЕ используем групповую валидацию по установкам — намеренно оцениваем упрощенный сценарий.


In [1]:
# Импорты
import os, sys, json
import numpy as np
import pandas as pd
from typing import List

# Визуализации опционально
import matplotlib.pyplot as plt
import seaborn as sns

# Данные
if '../src' not in sys.path:
    sys.path.append('../src')
from database import load_corrosion_data

# ML
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

print('Версии:', {'python': sys.version, 'numpy': np.__version__, 'pandas': pd.__version__, 'sklearn': sklearn.__version__})


Версии: {'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'numpy': '2.3.4', 'pandas': '2.3.3', 'sklearn': '1.7.2'}


### Про нули в химических признаках

- Ноль трактуем как «компонент отсутствует» — это нормальная и полезная информация для модели.
- Пропуски (NaN) — это «значение не измерено»; мы имитируем отсутствие компонент значением 0, чтобы модель могла их использовать. Для линейных моделей дополнительно масштабируем признаки.
- Подробнее про импутацию: https://scikit-learn.org/stable/modules/impute.html


In [2]:
# Загрузка и фильтр АВТ

df = load_corrosion_data()
print(f"Всего записей: {len(df):,}")

# Выберем поле(я) для фильтрации по строке "АВТ"
text_cols: List[str] = [c for c in ['installation'] if c in df.columns]
print('Поля для фильтра:', text_cols)

if not text_cols:
    raise ValueError('Нет текстовых полей для фильтрации АВТ')

mask = False
for c in text_cols:
    mask = mask | df[c].astype(str).str.contains('АВТ', case=False, na=False)

df_avt = df[mask].copy()
print(f"Отобрано АВТ: {len(df_avt):,}")

TARGET = 'corrosion_rate'
df_avt = df_avt[df_avt[TARGET].notna() & (df_avt[TARGET] >= 0) & (df_avt[TARGET] <= 10)]
print(f"После фильтрации цели: {len(df_avt):,}")


Всего записей: 442,052
Поля для фильтра: ['installation']
Отобрано АВТ: 233,306
После фильтрации цели: 177,841


In [6]:
# Формируем урезанный список признаков

base_features = [
    'cross_sectional_area',
    'material_resistance_score',
    'operating_temperature',
    'operating_pressure'
]

chemical_candidates = ['sulfur_content','total_sulfur_compounds','h2s_content','water_content','oxygen_content']

features = [c for c in base_features + chemical_candidates if c in df_avt.columns]
print('Признаки к использованию:', features)

X = df_avt[features].copy()
y = df_avt[TARGET].astype(float)

# Импутация: химические NaN -> 0 (нет данных ~ нет компонента), прочие NaN -> медиана
for c in features:
    if c in chemical_candidates:
        X[c] = X[c].fillna(0)
    else:
        X[c] = X[c].fillna(X[c].median())

print('Проверка пропусков после импутации:', int(X.isna().sum().sum()))


Признаки к использованию: ['cross_sectional_area', 'material_resistance_score', 'operating_temperature', 'operating_pressure', 'sulfur_content', 'total_sulfur_compounds', 'h2s_content', 'water_content', 'oxygen_content']
Проверка пропусков после импутации: 0


In [7]:
# Взаимодействия химия × температура

interaction_features = []
if 'operating_temperature' in X.columns:
    if 'water_content' in X.columns:
        name = 'water_content_x_operating_temperature'
        X[name] = X['water_content'] * X['operating_temperature']
        interaction_features.append(name)
    if 'oxygen_content' in X.columns:
        name = 'oxygen_content_x_operating_temperature'
        X[name] = X['oxygen_content'] * X['operating_temperature']
        interaction_features.append(name)

print('Добавлены взаимодействия:', interaction_features)


Добавлены взаимодействия: ['water_content_x_operating_temperature', 'oxygen_content_x_operating_temperature']


In [8]:
# Обучение: без групповой CV (train/val + KFold)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Ridge
ridge = Pipeline([('scaler', StandardScaler(with_mean=False)), ('model', Ridge(alpha=1.0, random_state=42))])
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_val)

ridge_mae = mean_absolute_error(y_val, ridge_pred)
ridge_rmse = np.sqrt(mean_squared_error(y_val, ridge_pred))
ridge_r2 = r2_score(y_val, ridge_pred)

print(f'Ridge(val): MAE={ridge_mae:.4f}, RMSE={ridge_rmse:.4f}, R2={ridge_r2:.4f}')

# RandomForest
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

rf_mae = mean_absolute_error(y_val, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
rf_r2 = r2_score(y_val, rf_pred)

print(f'RF(val):    MAE={rf_mae:.4f}, RMSE={rf_rmse:.4f}, R2={rf_r2:.4f}')

# KFold (без групп)
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)
rows = []
for fold, (tr, va) in enumerate(kf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr], X.iloc[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    r = Pipeline([('scaler', StandardScaler(with_mean=False)), ('model', Ridge(alpha=1.0, random_state=42))])
    r.fit(X_tr, y_tr)
    pr = r.predict(X_va)
    rows.append({'model':'Ridge','fold':fold,'MAE':mean_absolute_error(y_va, pr),'RMSE':np.sqrt(mean_squared_error(y_va, pr)),'R2':r2_score(y_va, pr)})

    rf_k = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
    rf_k.fit(X_tr, y_tr)
    prf = rf_k.predict(X_va)
    rows.append({'model':'RandomForest','fold':fold,'MAE':mean_absolute_error(y_va, prf),'RMSE':np.sqrt(mean_squared_error(y_va, prf)),'R2':r2_score(y_va, prf)})

cv_avt = pd.DataFrame(rows)
print('Средние метрики KFold:')
print(cv_avt.groupby('model')[['MAE','RMSE','R2']].agg(['mean','std']).round(4))


Ridge(val): MAE=0.0791, RMSE=0.2138, R2=0.0107
RF(val):    MAE=0.0762, RMSE=0.2038, R2=0.1014
Средние метрики KFold:
                 MAE            RMSE              R2        
                mean     std    mean     std    mean     std
model                                                       
RandomForest  0.0759  0.0005  0.2083  0.0047  0.0946  0.0104
Ridge         0.0788  0.0003  0.2178  0.0037  0.0107  0.0015


In [9]:
# Сохранение результатов

os.makedirs('../data', exist_ok=True)
summary = cv_avt.groupby('model')[['MAE','RMSE','R2']].agg(['mean','std']).round(6)
summary.columns = [f'{m}_{s}' for m, s in summary.columns]
cv_summary = summary.reset_index().to_dict(orient='records')

results = {
    'filtered_by': 'АВТ',
    'features_used': list(X.columns),
    'val_results': {
        'Ridge': {'MAE': float(ridge_mae), 'RMSE': float(ridge_rmse), 'R2': float(ridge_r2)},
        'RandomForest': {'MAE': float(rf_mae), 'RMSE': float(rf_rmse), 'R2': float(rf_r2)}
    },
    'cv_summary': cv_summary
}

with open('../data/ml_baseline_avt_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print('Сохранено: ../data/ml_baseline_avt_results.json')


Сохранено: ../data/ml_baseline_avt_results.json
